In [1]:
import json
import pandas as pd
import numpy as np

Sensor_readings = pd.read_json('data/json/W512.w512_readings (1).json')
Aircon_Data = pd.read_json('data/json/W512.w512_aircon_status (1).json')
Weather_readings = pd.read_json('data/json/user.weather_data (1).json')

def convert_AirconData(data):
    records = []
    
    for index, row in data.iterrows():
        # Parse FC_FullStatus_Readings if it's a string representation of a dictionary
        if isinstance(row['FC_FullStatus_Readings'], str):
            fc_readings = ast.literal_eval(row['FC_FullStatus_Readings'])
        else:
            fc_readings = row['FC_FullStatus_Readings']


        try:
            combined_datetime = pd.to_datetime(f"{row['date']} {row['time']}")
            formatted_datetime = pd.to_datetime(combined_datetime.strftime("%Y-%m-%d %H:%M:%S"))

        except Exception as e:
            print(f"Error combining datetime for row {index}: {e}")
            combined_datetime = None
            formatted_datetime = None

        
        # Create a record with base information
        record = {
            'Datetime': formatted_datetime
        }
        
        # Add each FC Unit's details as separate columns
        for unit, unit_data in fc_readings.items():
            record[f'{unit}_Status'] = unit_data['Status']
            record[f'{unit}_Fan_Status'] = unit_data['Fan_Status']
            record[f'{unit}_Set_Point'] = unit_data['Set_Point']
            record[f'{unit}_Operation_Mode'] = unit_data['Operation_Mode']
        
        records.append(record)
    
    # Create DataFrame
    df = pd.DataFrame(records)
    return df


def convert_sensorReadings(data):
    records = []
    
    # List of keys to exclude from Lorawan_Readings
    include_keys_1 = ["24E124725E285123", "24E124725E331695","24E124725E331744",
                      "24E124725E332483","24E124725E290348","24E124725E331733","24E124725E286745"]#"24E124136D316361" is suppiosed to be outdoor but it is not outdoor yet
    include_keys_2 = ["Sensor_1","Sensor_3","Sensor_6"]
    
    for index, row in data.iterrows():
        # Parse Energy_Readings if it's a string representation of a dictionary
        if isinstance(row['Energy_Readings'], str):
            Energy_readings = ast.literal_eval(row['Energy_Readings'])
        else:
            Energy_readings = row['Energy_Readings']
            
        # Parse Lorawan_Readings if it's a string representation of a dictionary
        if isinstance(row['Lorawan_Readings'], str):
            Lorawan_Readings = ast.literal_eval(row['Lorawan_Readings'])
        else:
            Lorawan_Readings = row['Lorawan_Readings']

        try:
            # Combine the date and time columns to create a datetime object
            combined_datetime = pd.to_datetime(f"{row['date']} {row['time']}")
            formatted_datetime = pd.to_datetime(combined_datetime.strftime("%Y-%m-%d %H:%M:%S"))
        except Exception as e:
            print(f"Error combining datetime for row {index}: {e}")
            formatted_datetime = None

        # Create a record with base information
        record = {
            'Datetime': formatted_datetime
        }
        
        # Add each Energy sensor's details as separate columns
        for unit, unit_data in Energy_readings.items():
            if unit not in include_keys_2:
                continue
                
            record[f'{unit}_Current'] = unit_data['Current']
            record[f'{unit}_Energy'] = unit_data['Energy']
            record[f'{unit}_Power'] = unit_data['Power']
        
        # Add each Lorawan device's details as separate columns
        for unit, unit_data in Lorawan_Readings.items():
            if unit not in include_keys_1:
                continue
            record[f'{unit}_Humidity'] = unit_data.get('humidity', None)
            record[f'{unit}_Temperature'] = unit_data.get('temperature', None)

            co2_value = unit_data.get('co2', None)
            if co2_value is not None:
                record[f'{unit}_CO2'] = co2_value

        # Append the record to the list of records
        records.append(record)
    df=pd.DataFrame(records)
    return df


def convert_weatherData(data):
    records = []
    for index, row in data.iterrows():
        # Parse Energy_Readings if it's a string representation of a dictionary
        if isinstance(row['result'], str):
            weather_results = ast.literal_eval(row['result'])
        else:
            weather_results = row['result']
            
        try:
            combined_datetime = pd.to_datetime(f"{row['date']} {row['time']}")
            formatted_datetime = pd.to_datetime(combined_datetime.strftime("%Y-%m-%d %H:%M:%S"))
        except Exception as e:
            print(f"Error combining datetime for row {index}: {e}")
            formatted_datetime = None

        record = {
            'Datetime': formatted_datetime
        }  

        record['weather_status'] = weather_results['weather_status']
        record['weather_temp'] = weather_results['weather_temp']
        record['weather_humid'] = weather_results['weather_humidity']
            
        records.append(record)
    df=pd.DataFrame(records)
    return df



Aircon_data_df = convert_AirconData(Aircon_Data)
Aircon_data_df = Aircon_data_df[3194:]
Sensor_readings_df = convert_sensorReadings(Sensor_readings)
Sensor_readings_df = Sensor_readings_df.interpolate(method='linear')
weather_readings_df = convert_weatherData(Weather_readings)

# Merge Aircon data with Sensor readings using merge_asof
merged_df = pd.merge_asof(Aircon_data_df, Sensor_readings_df, on='Datetime', direction='nearest')

# Now, merge the Weather readings with the previous result using merge_asof
merged_df = pd.merge_asof(merged_df, weather_readings_df, on='Datetime', direction='nearest')


merged_df['total_energy'] = (
    merged_df['Sensor_1_Energy'] +
    merged_df['Sensor_3_Energy'] +
    merged_df['Sensor_6_Energy']
)

merged_df['total_power'] = (
    merged_df['Sensor_1_Power'] +
    merged_df['Sensor_3_Power'] +
    merged_df['Sensor_6_Power']
)

merged_df['total_current'] = (
    merged_df['Sensor_1_Current'] +
    merged_df['Sensor_3_Current'] +
    merged_df['Sensor_6_Current']
)

temperature_col = [
    col for col in merged_df.columns 
    if "24e124" in col.lower() and "temperature" in col.lower()
]
humidity_col = [
    col for col in merged_df.columns 
    if "24e124" in col.lower() and "humidity" in col.lower()
]
co2_col = [
    col for col in merged_df.columns 
    if "24e124" in col.lower() and "co2" in col.lower()
]

merged_df['avg_temperature'] = merged_df[temperature_col].mean(axis=1)
merged_df['avg_humidity'] = merged_df[humidity_col].mean(axis=1)
merged_df['avg_co2'] = merged_df[co2_col].mean(axis=1)

# merged_df['future_timestamp'] = merged_df['Datetime'] + pd.Timedelta(minutes=30)

# merged_df['target_temperature'] = pd.merge_asof(
#     merged_df[['future_timestamp']].rename(columns={'future_timestamp': 'Datetime'}),
#     merged_df[['Datetime', 'avg_temperature']],
#     on='Datetime',
#     direction='nearest'
# )['avg_temperature']

# merged_df['target_humidity'] = pd.merge_asof(
#     merged_df[['future_timestamp']].rename(columns={'future_timestamp': 'Datetime'}),
#     merged_df[['Datetime', 'avg_humidity']],
#     on='Datetime',
#     direction='nearest'
# )['avg_humidity']

# merged_df['future_energy'] = pd.merge_asof(
#     merged_df[['future_timestamp']].rename(columns={'future_timestamp': 'Datetime'}),
#     merged_df[['Datetime', 'total_energy']],
#     on='Datetime',
#     direction='nearest'
# )['total_energy']

# merged_df['energy_difference'] = merged_df['future_energy'] - merged_df['total_energy']


dropped_col = [
    col for col in merged_df.columns
    if "24e124" in col.lower()
]
dropped_col_sensor = [
    col for col in merged_df.columns
    if "sensor" in col.lower()
]

columns_to_drop = dropped_col + dropped_col_sensor
merged_df.drop(columns=columns_to_drop, axis=1, inplace=True)



merged_df.info()






<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2438 entries, 0 to 2437
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Datetime                  2438 non-null   datetime64[ns]
 1   FC_Unit_1_Status          2438 non-null   object        
 2   FC_Unit_1_Fan_Status      2438 non-null   object        
 3   FC_Unit_1_Set_Point       2438 non-null   float64       
 4   FC_Unit_1_Operation_Mode  2438 non-null   object        
 5   FC_Unit_2_Status          2438 non-null   object        
 6   FC_Unit_2_Fan_Status      2438 non-null   object        
 7   FC_Unit_2_Set_Point       2438 non-null   float64       
 8   FC_Unit_2_Operation_Mode  2438 non-null   object        
 9   FC_Unit_3_Status          2438 non-null   object        
 10  FC_Unit_3_Fan_Status      2438 non-null   object        
 11  FC_Unit_3_Set_Point       2438 non-null   float64       
 12  FC_Unit_3_Operation_

In [ ]:
from tqdm import tqdm

min_energy = merged_df['total_energy'].min()
max_energy = merged_df['total_energy'].max()
min_temp = merged_df['avg_temperature'].min()
max_temp = merged_df['avg_temperature'].max()

energy_weight = 0.5
temp_weight = 0.5

# Initialize variables
current_temperature = 22
target_temperature = 24
best_path = []
min_cost = float('inf')
best_cost, best_path = float('inf'), []


# Memoization dictionary
memo = {}


def calculate_differences(row1, row2):
    energy_diff = (abs(row1['total_energy'] - row2['total_energy']) - min_energy) / (max_energy - min_energy)
    temp_diff = (abs((row1['avg_temperature'] + row2['avg_temperature']) / 2 - target_temperature) - min_temp) / (max_temp - min_temp)
    return energy_diff, temp_diff



def find_best_pathway(current_row_index, current_temp, path, total_cost):
    # Use a tuple of (current_row_index, current_temp) as the key for memoization
    key = (current_row_index, tuple(path))
    if key in memo:
        return memo[key]

    # Base case
    if abs(current_temp - target_temperature) < 0.3:
        return total_cost, path[:]

    best_cost = float('inf')
    best_path = None

    for next_row_index in range(current_row_index + 1, len(merged_df)):
        next_row = merged_df.iloc[next_row_index]
        energy_diff, temp_diff = calculate_differences(merged_df.iloc[current_row_index], next_row)
        cost = energy_diff * energy_weight + temp_diff * temp_weight

        if total_cost + cost >= min_cost:
            continue

        # Recursive call
        new_cost, new_path = find_best_pathway(
            next_row_index, 
            next_row['avg_temperature'], 
            path + [next_row_index], 
            total_cost + cost
        )

        if new_cost < best_cost:
            best_cost = new_cost
            best_path = new_path

    # Store result in memo
    memo[key] = (best_cost, best_path)
    return memo[key]


for start_index, row in tqdm(merged_df.iterrows(), total=len(merged_df), desc="Exploring paths"):
    if abs(row['avg_temperature'] - current_temperature) <= 0.5:
        cost, path = find_best_pathway(start_index, row['avg_temperature'], [start_index], 0)
        if cost < best_cost:
            best_cost = cost
            best_path = path

# Output the result
if best_path:
    print("Best Path:", best_path)
    print("Minimum Cost:", best_cost)
else:
    print("No valid pathway found.")


In [21]:
import torch

# Convert DataFrame columns to PyTorch tensors
device = 'cuda' if torch.cuda.is_available() else 'cpu'

energy = torch.tensor(merged_df['total_energy'].values, device=device, dtype=torch.float32)
temperature = torch.tensor(merged_df['avg_temperature'].values, device=device, dtype=torch.float32)

# Convert Datetime column to seconds and move to GPU
datetime_seconds = torch.tensor(
    (merged_df['Datetime'] - merged_df['Datetime'].min()).dt.total_seconds().values,
    device=device,
    dtype=torch.float32
)

# Precompute min and max for normalization
min_energy = energy.min()
max_energy = energy.max()
min_temp = temperature.min()
max_temp = temperature.max()
min_time = datetime_seconds.min()
max_time = datetime_seconds.max()

# Weights for the cost function
energy_weight = 0.3
temp_weight = 0.3
time_weight = 0.4

#this is where you edit your variables
current_temperature = 25.0
target_temperature = 22.0
best_path = []
min_cost = float('inf')

# Memoization dictionary
memo = {}


def calculate_differences_gpu(row1_energy, row1_temp, row1_time, row2_energy, row2_temp, row2_time):
    # Calculate energy and temperature differences
    energy_diff = (torch.abs(row1_energy - row2_energy) - min_energy) / (max_energy - min_energy)
    temp_diff = (torch.abs((row1_temp + row2_temp) / 2 - target_temperature) - min_temp) / (max_temp - min_temp)
    # Calculate normalized time difference
    time_diff = (torch.abs(row1_time - row2_time) - min_time) / (max_time - min_time)
    return energy_diff, temp_diff, time_diff


def find_best_pathway(current_row_index, current_temp, path, total_cost, time_taken):
    global min_cost, best_path

    # Memoization key
    key = (current_row_index, tuple(path))
    if key in memo:
        return memo[key]

    # Base case: Target temperature reached
    if abs(current_temp - target_temperature) < 0.3:
        if total_cost < min_cost:
            min_cost = total_cost
            best_path = path[:]
        return total_cost, path[:], time_taken

    best_cost = float('inf')
    best_path_local = None

    for next_row_index in range(current_row_index + 1, len(energy)):
        # Calculate differences using tensors
        energy_diff, temp_diff, time_diff = calculate_differences_gpu(
            energy[current_row_index],
            temperature[current_row_index],
            datetime_seconds[current_row_index],
            energy[next_row_index],
            temperature[next_row_index],
            datetime_seconds[next_row_index]
        )

        # Compute cost
        cost = energy_diff * energy_weight + temp_diff * temp_weight + time_diff * time_weight

        # Pruning condition
        if total_cost + cost >= min_cost:
            continue

        # Recursive call
        new_cost, new_path, new_time_taken = find_best_pathway(
            next_row_index,
            temperature[next_row_index].item(),
            path + [next_row_index],
            total_cost + cost,
            time_taken + (datetime_seconds[next_row_index] - datetime_seconds[current_row_index]).item()
        )

        # Update best cost and path
        if new_cost < best_cost:
            best_cost = new_cost
            best_path_local = new_path

    # Store result in memo
    memo[key] = (best_cost, best_path_local, time_taken)
    return memo[key]


# Start the backtracking process
for start_index in range(len(energy)):
    if abs(temperature[start_index].item() - current_temperature) <= 0.5:
        print(f"Starting backtracking from index {start_index} with temperature {temperature[start_index].item()}")
        find_best_pathway(start_index, temperature[start_index].item(), [start_index], 0, 0)

# Output the result


fcunit_col = [
    col for col in merged_df.columns 
    if "fc_unit" in col.lower()
]
if best_path:
    print("Best Path:", best_path)
    print("Minimum Cost:", min_cost)
    
    # Convert time taken to minutes
    time_taken_in_minutes = sum([datetime_seconds[best_path[i]] - datetime_seconds[best_path[i-1]] for i in range(1, len(best_path))]).item() / 60
    print(f"Time Taken (in minutes): {time_taken_in_minutes:.2f}min to go from {current_temperature} to {target_temperature}")

    best_path_rows = merged_df.iloc[best_path][fcunit_col+['avg_temperature']]
    
    best_path_rows.to_csv(f'{current_temperature}_to_{target_temperature}.csv')
else:
    print("No valid pathway found.")


Starting backtracking from index 8 with temperature 24.657142639160156
Starting backtracking from index 9 with temperature 24.842857360839844
Starting backtracking from index 10 with temperature 25.0
Starting backtracking from index 11 with temperature 25.14285659790039
Starting backtracking from index 52 with temperature 24.5
Starting backtracking from index 53 with temperature 24.614286422729492
Starting backtracking from index 54 with temperature 24.685714721679688
Starting backtracking from index 55 with temperature 24.785715103149414
Starting backtracking from index 56 with temperature 24.842857360839844
Starting backtracking from index 57 with temperature 24.89285659790039
Starting backtracking from index 58 with temperature 24.985713958740234
Starting backtracking from index 59 with temperature 25.035715103149414
Starting backtracking from index 60 with temperature 25.071428298950195
Starting backtracking from index 61 with temperature 25.171428680419922
Starting backtracking fr